In [ ]:
docker run -d /
    --name elasticsearch  /
    -p 9200:9200 /
    -e "discovery.type=single-node" /
    -e "xpack.security.enabled=false" /
    -e "xpack.security.transport.ssl.enabled=false" /
    -v es_data:/usr/share/elasticsearch/data /
    docker.elastic.co/elasticsearch/elasticsearch:8.10.0

In [1]:
from elasticsearch import Elasticsearch

# Connexion locale
es = Elasticsearch("http://localhost:9200")

# Vérifier la connexion
if es.ping():
    print("Connecté à Elasticsearch ✅")
else:
    print(es)
    raise ValueError("Connexion à Elasticsearch échouée ❌")


Connecté à Elasticsearch ✅


In [6]:
import tarfile
import io
from lxml import etree

nom_fichier_taz = "../data/raw/AN_2022001.taz"

with tarfile.open(nom_fichier_taz, "r:*") as taz:
    membre_tar = next(m for m in taz.getmembers() if m.name.endswith(".tar"))
    tar_bytes = taz.extractfile(membre_tar).read()
    tar_buffer = io.BytesIO(tar_bytes)

    with tarfile.open(fileobj=tar_buffer, mode="r:") as tar:
        for membre in tar.getmembers():
            if membre.name.endswith(".xml"):
                f = tar.extractfile(membre)
                if f is None:
                    continue

                xml_bytes = f.read()
                root = etree.fromstring(xml_bytes)

                contenu = root.find(".//ContenuDANBleu")
                if contenu is None:
                    contenu = root.find(".//ContenuDANBlanc")

                if contenu is None:
                    continue

                print(len(contenu.xpath(".//intervention")), "interventions dans", membre.name)
                for inter in contenu.xpath(".//intervention"):
                    orateur = inter.xpath(".//orateur/@nom")
                    groupe = inter.xpath(".//orateur/@groupe")
                    texte = " ".join(inter.xpath(".//texte//text()"))

                    doc = {
                        "fichier_xml": membre.name,
                        "orateur": orateur[0] if orateur else None,
                        "groupe": groupe[0] if groupe else None,
                        "texte": texte
                    }

                    # Envoyer dans Elasticsearch
                    res = es.index(index="deputes_interventions", document=doc)
                    print(f"Document indexé : {res['_id']}")


0 interventions dans CRI_20220104_001.xml
0 interventions dans AAA_20220104_001.xml
